In [14]:
import pandas as pd
import numpy as np
import statistics as sta
import warnings
warnings.filterwarnings('ignore')

In [15]:
demandRights1 = pd.read_csv('data/new/DemandRights01.csv')
demandRights2 = pd.read_csv('data/new/DemandRights02.csv')
#APPEND 3 DF 
demandRights = demandRights1.append([demandRights2], ignore_index=True)

countriesCode = pd.read_csv('data/countriesCode.csv')

#RENAME COLUMNS
countriesCode.columns = ['Country','CountryCode']
demandRights.rename(columns = {'SQLDATE':'Date','Actor1CountryCode':'CountryCode','SOURCEURL':'SourceURL','avgTone':'AvgTone'}, inplace=True)

#REMOVE SPACE
countriesCode['CountryCode'] = countriesCode['CountryCode'].str.strip()
countriesCode['Country'] = countriesCode['Country'].str.strip()

In [16]:
demandRights.head()

,Date,CountryCode,AvgTone,SourceURL
0,20180101,USA,-1.699717,http://www.cope.es/noticias/actualidad/2018-an...
1,20180101,IRN,-4.892966,http://www.alwast.net/world-news/article-2196081
2,20180101,MEX,-3.743316,http://ciudadania-express.com/2018/01/01/orden...
3,20180101,IRN,-10.981697,http://www.dailymail.co.uk/wires/afp/article-5...
4,20180101,IRN,-4.892966,http://www.alwast.net/world-news/article-2196081


In [17]:
len(demandRights)

16112

In [18]:
demandRights.duplicated

<bound method DataFrame.duplicated of            Date CountryCode    AvgTone  \
0      20180101         USA  -1.699717   
1      20180101         IRN  -4.892966   
2      20180101         MEX  -3.743316   
3      20180101         IRN -10.981697   
4      20180101         IRN  -4.892966   
5      20180101         IRN  -4.615385   
6      20180101         DNK  -1.083032   
7      20180101         IRN  -4.892966   
8      20180101         IRN  -4.892966   
9      20180101         IRN  -4.615385   
10     20180101         UKR  -5.369128   
11     20180101         ESP  -1.361868   
12     20180101         ISR  -3.180212   
13     20180101         PSE  -5.208333   
14     20180101         USA   1.128936   
15     20180101         USA   1.128936   
16     20180101         FRA  -6.127451   
17     20180101         FRA  -6.127451   
18     20180101         SSD  -4.931507   
19     20180101         JPN  -5.749293   
20     20180101         USA   1.128936   
21     20180101         ISR  -3.180212

In [19]:
#drop data duplicated
demandRightsDropDup = demandRights.drop_duplicates()
len(demandRightsDropDup)

11644

## MERGE DATA WITH COUNTRY CODE

In [20]:
demandRightsData = demandRightsDropDup.merge(countriesCode, on='CountryCode', how='left')
demandRightsData.info()
isnull = demandRightsData[demandRightsData.isnull().any(axis=1)]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11644 entries, 0 to 11643
Data columns (total 5 columns):
Date           11644 non-null int64
CountryCode    11644 non-null object
AvgTone        11644 non-null float64
SourceURL      11644 non-null object
Country        11199 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 545.8+ KB


In [21]:
isnull.groupby('CountryCode').count()

,Date,AvgTone,SourceURL,Country
CountryCode,,,,
AFR,135,135,135,0
EUR,250,250,250,0
MEA,1,1,1,0
NMR,1,1,1,0
SAS,1,1,1,0
SEA,1,1,1,0
SSD,7,7,7,0
TWN,17,17,17,0
WAF,2,2,2,0


## MERGE DATA WITH ACTOR CODE

In [22]:
actorCodeData = pd.read_csv('data/actorCode.csv', delimiter = ',', encoding = "ISO-8859-1")
#RENAME COLUMNS
actorCodeData.columns = ['actorCode','Actor']
#REMOVE SPACE
actorCodeData['actorCode'] = actorCodeData['actorCode'].str.strip()
actorCodeData['Actor'] = actorCodeData['Actor'].str.strip()

demandRightsDatan = demandRightsData.merge(actorCodeData, left_on='CountryCode', right_on='actorCode', how='left')

#cek pengaruh MERGE DATA WITH ACTOR CODE
demandRightsDatan[(demandRightsDatan.Actor.notnull())&(demandRightsDatan.Country.isnull())]


,Date,CountryCode,AvgTone,SourceURL,Country,actorCode,Actor
17,20180101,EUR,1.775523,http://www.americasquarterly.org/content/two-t...,NaN,EUR,Europe
49,20180103,EUR,-5.333333,https://dnews.dn.ua/news/662191,NaN,EUR,Europe
51,20180103,EUR,-3.076923,http://www.lepoint.fr/monde/face-aux-manifesta...,NaN,EUR,Europe
52,20180103,EUR,-3.109168,http://www.franceguyane.fr/actualite/france/fa...,NaN,EUR,Europe
56,20180103,EUR,-3.155007,http://www.franceguyane.fr/actualite/france/fa...,NaN,EUR,Europe
66,20180103,EUR,-5.670103,http://www.paneuropeannetworks.com/government/...,NaN,EUR,Europe
94,20180104,EUR,-3.338392,https://www.myt.mu/sinformer/afp/actualites/49...,NaN,EUR,Europe
112,20180105,AFR,-3.524426,http://newlaborforum.cuny.edu/2018/01/05/the-m...,NaN,AFR,Africa
125,20180105,EUR,1.226994,http://www.elbalad.news/3110223,NaN,EUR,Europe
210,20180108,EUR,-0.416667,https://ria.ru/economy/20180108/1512244694.html,NaN,EUR,Europe


In [23]:
#INTERNATIONAL REGION CODES
# demandRightsData.loc[demandRightsData['CountryCode'] == 'AFR', 'Country'] = 'Africa'
# demandRightsData.loc[demandRightsData['CountryCode'] == 'EUR', 'Country'] = 'Europe'
# demandRightsData.loc[demandRightsData['CountryCode'] == 'WST', 'Country'] = 'the West'
# demandRightsData.loc[demandRightsData['CountryCode'] == 'SSD', 'Country'] = 'South Sudan'
# demandRightsData.loc[demandRightsData['CountryCode'] == 'WAF', 'Country'] = 'West Africa'
# demandRightsData.loc[demandRightsData['CountryCode'] == 'SAS', 'Country'] = 'South Asia'
# demandRightsData.loc[demandRightsData['CountryCode'] == 'NMR', 'Country'] = 'North America'
# demandRightsData.loc[demandRightsData['CountryCode'] == 'MEA', 'Country'] = 'Middle East'
# demandRightsData.loc[demandRightsData['CountryCode'] == 'SEA', 'Country'] = 'Southeast Asia'
demandRightsData.loc[demandRightsData['CountryCode'] == 'TWN', 'Country'] = 'Taiwan'

demandRightsData.dropna(inplace=True)
demandRightsData[(demandRightsData.isnull().any(axis=1))]

,Date,CountryCode,AvgTone,SourceURL,Country


In [24]:
#demandRightsData[demandRightsData.isnull().any(axis=1)]

# DEVELOPING/DEVELOPED COUNTRY

In [25]:
devC = pd.read_csv('data/developingCountries.csv', delimiter = ',', encoding = "ISO-8859-1")
devC['CountryCat'] = 'developing'
devC.drop('Country', axis =1, inplace =True)
devC.dropna(inplace=True)

demandRights = demandRightsData.merge(devC, on=['CountryCode'], how='left')
demandRights.loc[demandRights.CountryCode.notnull(), 'CountryCat'] = demandRights.loc[demandRights.CountryCode.notnull(), 'CountryCat'].fillna('developed')
demandRights['HRTopic'] = 'demand rights'
demandRights = demandRights[['Date','Country','CountryCode','CountryCat','HRTopic','AvgTone', 'SourceURL']]
demandRights

,Date,Country,CountryCode,CountryCat,HRTopic,AvgTone,SourceURL
0,20180101,United States,USA,developed,demand rights,-1.699717,http://www.cope.es/noticias/actualidad/2018-an...
1,20180101,Iran,IRN,developing,demand rights,-4.892966,http://www.alwast.net/world-news/article-2196081
2,20180101,Mexico,MEX,developing,demand rights,-3.743316,http://ciudadania-express.com/2018/01/01/orden...
3,20180101,Iran,IRN,developing,demand rights,-10.981697,http://www.dailymail.co.uk/wires/afp/article-5...
4,20180101,Iran,IRN,developing,demand rights,-4.615385,https://www.hespress.com/international/376637....
5,20180101,Denmark,DNK,developed,demand rights,-1.083032,https://www.information.dk/indland/2018/01/tid...
6,20180101,Ukraine,UKR,developing,demand rights,-5.369128,http://day.kyiv.ua/uk/news/010118-u-kyyevi-vym...
7,20180101,Spain,ESP,developed,demand rights,-1.361868,http://www.subrayado.com.uy/noticias/73715/pui...
8,20180101,Israel,ISR,developed,demand rights,-3.180212,https://www.rt.com/news/414734-israel-annexati...
9,20180101,Occupied Palestinian Territory,PSE,developed,demand rights,-5.208333,https://www.alwatanvoice.com/arabic/news/2017/...


In [26]:
#demandRights.to_csv('demandRightsData.csv', index=False)
#before: 11674
demandRights.to_csv('MergingData/DataDemandRights.csv',index=False)